[신경망 모델 구성하기](https://tutorials.pytorch.kr/beginner/basics/buildmodel_tutorial.html)

In [1]:
"""
신경망은 데이터에 대한 연산을 수행하는 계층(모듈)로 구성

torch.nn: 신경망을 구성하는데 필요한 모든 구성 요소를 제공
중첩된 구조를 이용하여 복잡한 아키텍처를 쉽게 구축하고 관리
"""

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 학습을 위한 장치 얻기

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


## 클래스 정의하기

In [6]:
"""
신경망 모델을 nn.Module의 하위클래스로 정의
__init__ 에서 신경망 계층들을 초기화
nn.Module을 상속받은 모든 클래스는 foraward 메소드에 연산들을 구현
"""

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
# NeuralNetwork의 인스턴스를 생성하고 device로 이동, 구조를 출력
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
"""
모델 사용을 위해 입력 데이터 전달
    모델의 forward를 실행하ㅐ게 된다

모델에 입력 전달하여 호출하면 2차원 텐서 반환
    2차원 텐서의 dim=0은 각 분류에 대한 예측값 10개
    dim=1에는 각 출력의 개별 값들이 해당
    -> 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜서 예측 확률을 얻는다
"""

X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6])


## 모델 계층

In [9]:
"""
모델 계층 살펴보기

예) 28*28 크기의 이미지 3개로 구성된 미니배치를 이용해서
    신경망을 통과할 때 어떤 일이 발생하는지 확인하기
"""

input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [12]:
# nn.Flatten
# 각 28*28 의 2D 이미지를 784 픽셀 값을 가지는 연속된 배열로 변환
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [13]:
# nn.Linear
# 저장된 W, b 이용하여 입력에 선형 변환 적용
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [14]:
# nn.ReLU
# 비선형성 도입, 신경망이 다양한 현상을 학습하도록 돕는다
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-1.5752e-02, -6.7736e-01,  4.2277e-01,  6.8610e-01,  1.5518e-01,
          2.2709e-01, -5.6696e-01,  2.0519e-01,  9.8302e-02, -1.5035e-01,
          2.5006e-02,  2.1959e-01,  8.3050e-02,  3.3268e-01, -6.1633e-04,
          6.8422e-01,  3.6322e-01,  6.5807e-01, -1.3880e-01,  2.6352e-01],
        [ 6.1222e-02, -5.3631e-01,  1.1278e-01,  6.5239e-01, -2.7254e-02,
          3.9589e-01, -7.7126e-01,  4.4474e-01,  1.4641e-01, -5.5568e-02,
         -1.5782e-01,  1.0911e-01,  7.8375e-02,  3.2912e-01,  1.8765e-01,
          8.8016e-01, -2.9660e-02,  7.6310e-01, -2.2078e-01, -2.3172e-02],
        [-8.2940e-02, -6.5397e-01,  2.0403e-01,  6.7359e-01, -8.5573e-02,
          3.7883e-01, -6.9567e-01,  1.0985e-01, -4.3660e-02,  7.7138e-02,
         -2.6001e-01,  1.1416e-01,  3.4738e-01,  2.0507e-01, -1.0303e-01,
          6.9274e-01,  2.1906e-01,  7.8075e-01, -2.9755e-01, -1.4580e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.4228, 0.6861, 0.1552,

In [15]:
# nn.Sequential
# 순서를 갖는 모듈의 컨테이너
# 데이터가 저장된 것과 같은 순서로 모든 모듈로 전달된다
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [16]:
# nn.Softmax
# logits 반환, 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정된다
# dim 매개변수는 값의 합이 1이 되는 차원
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [17]:
# 모델 매개변수
# nn.Module 상속시에는 모델 객체 내부의 모든 필드들이 자동 추적됨
# 모델의 parameters(), named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 된다

print(f"Model structure: {model}\n\n")

# 각 매개변수들을 순회하며 매개변수의 크기와 값을 출력
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0117, -0.0262,  0.0219,  ..., -0.0289,  0.0219,  0.0202],
        [-0.0329,  0.0171,  0.0020,  ..., -0.0026,  0.0082,  0.0217]],
       grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0113, -0.0027], grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0091, -0.0222,  0.0245,  ...,  0.0148,  0.0024,  0.0359],
        [ 0.0342, -0.0053, -0.0436,  ..., -0.0265,  0.0090,  0.0326]],
       grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.bias | Size: 